# Python for Algorithmic Trading 

## Visualizing Streaming Data with Plotly

#### To follow along, the plotly package needs to be installed. Also, several Jupyter Lab extensions need to be installed when working with Jupyter Lab. The following command should be executed on the terminal:

```
conda install plotly ipywidgets
jupyter labextension install jupyterlab-plotly
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install plotlywidget
```

### The Basics

In [1]:
import zmq
from datetime import datetime
import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')

In [2]:
symbol = 'SYMBOL'

In [3]:
fig = go.FigureWidget()
fig.add_scatter()
fig

FigureWidget({
    'data': [{'type': 'scatter', 'uid': '7ea875d6-360d-4bef-b3f5-591e3b0b0e24'}], 'layout': {'t…

In [4]:
context = zmq.Context()

In [5]:
socket = context.socket(zmq.SUB)

In [6]:
socket.connect('tcp://0.0.0.0:5555')

In [7]:
socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [8]:
times = list()
prices = list()

In [9]:
for _ in range(50):
    msg = socket.recv_string()
    t = datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices

In [10]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),
                mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'),
                mode='lines+markers')
fig

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': 'de3bb6f3-1a9a-4525-9290-0890b9a3e564…

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame()

### A streaming plot with Plotly can have multiple graph objects. This comes in handy when, for instance, two simple moving averages (SMAs) shall be visualized in real time in addition to the price ticks. The following code instantiates again a figure widget—this time with three scatter objects. The tick data from the sample tick data server is collected in a pandas DataFrame object. The two SMAs are calculated after each update from the socket. The amended data sets are used to update the data object of the figure widget

In [13]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']

### As with conventional Plotly plots, streaming plots based on figure widgets can also have multiple sub-plots. The example that follows creates a streaming plot with three sub-plots. The first plots the real-time tick data. The second plots the log returns data. The third plots the time series momentum based on the log returns data.

In [14]:
from plotly.subplots import make_subplots

In [15]:
f = make_subplots(rows=3, cols=1, shared_xaxes=True)
f.append_trace(go.Scatter(name='SYMBOL'), row=1, col=1)
f.append_trace(go.Scatter(name='RETURN', line=dict(width=1, dash='dot'),
                mode='lines+markers', marker={'symbol': 'triangle-up'}),
                row=2, col=1)
f.append_trace(go.Scatter(name='MOMENTUM', line=dict(width=1, dash='dash'),
                mode='lines+markers', marker={'symbol': 'x'}), row=3, col=1)
# f.update_layout(height=600)

In [16]:
fig = go.FigureWidget(f)

In [17]:
fig

FigureWidget({
    'data': [{'name': 'SYMBOL',
              'type': 'scatter',
              'uid': '5b4c3dd8…

In [18]:
import numpy as np

In [19]:
df = pd.DataFrame()

In [20]:
for _ in range(75):
    msg = socket.recv_string()
    t = datetime.now()
    sym, price = msg.split()
    df = df.append(pd.DataFrame({sym: float(price)}, index=[t]))
    df['RET'] = np.log(df[sym] / df[sym].shift(1))
    df['MOM'] = df['RET'].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['RET']
    fig.data[2].y = df['MOM']

In [21]:
socket = context.socket(zmq.SUB)

In [22]:
socket.connect('tcp://0.0.0.0:5556')

In [23]:
socket.setsockopt_string(zmq.SUBSCRIBE, '')

In [24]:
for _ in range(5):
    msg = socket.recv_string()
    print(msg)

43.685 44.893 88.456 40.969 66.011 81.481 23.774 35.585
50.248 61.707 8.052 35.117 70.761 9.553 6.560 63.855
19.268 19.866 12.338 57.461 47.577 6.090 12.851 78.318
35.071 33.976 27.117 68.415 48.806 14.524 0.832 79.783
24.542 94.013 92.181 3.378 69.987 29.676 13.989 13.699


In [25]:
fig = go.FigureWidget()
fig.add_bar()
fig

FigureWidget({
    'data': [{'type': 'bar', 'uid': 'ddf5fbfd-1c9e-4903-9a6f-488f91a0e895'}], 'layout': {'templ…

In [ ]:
x = list('abcdefgh')
fig.data[0].x = x
for _ in range(100):
    msg = socket.recv_string()
    y = msg.split()
    y = [float(n) for n in y]
    fig.data[0].y = y